In [1]:
from fastai.data.all import *
from fastai.learner import *
from fastai.losses import CrossEntropyLossFlat
from fastai.optimizer import Adam, OptimWrapper, params
from fastai.metrics import accuracy, F1Score
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers import AutoModelForSequenceClassification
import pandas as pd
from blurr.text.data.core import *
from blurr.text.modeling.core import *
from blurr.text.utils import NLP as BLURR # lo importamos así para que sea compatible con el resto
from datasets import load_dataset, concatenate_datasets

/home/jonathan/.virtualenvs/huggingface/lib/python3.6/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
import torch
torch.cuda.set_device(0)

In [3]:
train_df = pd.read_csv('perplejidad/datasets/train_train_set.csv',delimiter='\t')

In [4]:
train_df['is_valid']=False

In [5]:
dl_kwargs = {"bs": 64, "val_bs": 64}
learn_kwargs = {"metrics": [accuracy]}

In [6]:
test_df = pd.read_csv('perplejidad/datasets/test_set.csv',delimiter='\t')

In [7]:
test_df['is_valid']=True

In [8]:
train_test_df = pd.concat([train_df,test_df])

In [9]:
train_idxs, test_idxs = L(range(len(train_test_df[train_test_df.is_valid==False]))), L(range(len(train_test_df[train_test_df.is_valid==False]), len(train_test_df)))

In [10]:
learnXLMRoberta = BlearnerForSequenceClassification.from_data(
    train_test_df,
    "xlm-roberta-base",
    text_attr="Sentence",
    label_attr="Level",
    dblock_splitter=IndexSplitter(test_idxs),
    dl_kwargs=dl_kwargs,
    learner_kwargs=learn_kwargs,
)

In [11]:
learnXLMRoberta.load('xlm-roberta-base')

In [13]:
dfCEDEL = pd.read_csv('cedelSample.csv')

In [14]:
def predict(text):    
    prediction = learnXLMRoberta.predict(text)[0]
    return prediction['class_labels'][prediction['class_index']]
    
dfCEDEL['prediction'] = dfCEDEL['Text'].apply(lambda x : predict(x))

In [15]:
import numpy as np
niveles = ['A1','A2','B1','B2','C1']

def predict_sentences(text):
    sentences = text.split('.')
    prediction = np.sum([learnXLMRoberta.predict(x)[0]['probs'] for x in sentences],axis=0)/len(sentences)
    return niveles[np.argmax(prediction)]
    
dfCEDEL['predictionSencences'] = dfCEDEL['Text'].apply(lambda x : predict_sentences(x))

In [16]:
import numpy as np
niveles = ['A1','A2','B1','B2','C1']

def predict_sentences_split(text):
    sentences = text.split('.')
    prediction = [niveles[np.argmax(learnXLMRoberta.predict(x)[0]['probs'])] for x in sentences]
    return prediction
    
dfCEDEL['predictionIndSencences'] = dfCEDEL['Text'].apply(lambda x : predict_sentences_split(x))

In [17]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [18]:
accuracy_score(dfCEDEL['level'],dfCEDEL['prediction'])

0.26

In [19]:
accuracy_score(dfCEDEL[dfCEDEL.level!='C2']['level'],dfCEDEL[dfCEDEL.level!='C2']['prediction'])

0.312

In [20]:
accuracy_score(dfCEDEL[dfCEDEL.level!='C2']['level'],dfCEDEL[dfCEDEL.level!='C2']['predictionSencences'])

0.328

In [21]:
confusion_matrix(dfCEDEL[dfCEDEL.level!='C2']['level'],dfCEDEL[dfCEDEL.level!='C2']['prediction'])

array([[14, 24,  5,  2,  5],
       [ 9, 26,  3,  5,  7],
       [ 4, 20,  7, 11,  8],
       [ 1, 18,  7,  8, 16],
       [ 0, 13,  5,  9, 23]])

In [22]:
confusion_matrix(dfCEDEL[dfCEDEL.level!='C2']['level'],dfCEDEL[dfCEDEL.level!='C2']['predictionSencences'])

array([[20, 18,  6,  2,  4],
       [15, 22,  5,  1,  7],
       [ 8, 18,  9,  3, 12],
       [ 1, 17,  8,  7, 17],
       [ 0, 10, 10,  6, 24]])

In [23]:
dfCEDELNoC2 = dfCEDEL[dfCEDEL.level!='C2']

In [27]:
gt = []
preds = []

for level,predictions in zip(dfCEDELNoC2.level,dfCEDELNoC2.predictionIndSencences):
    for x in predictions:
        gt.append(level)
        preds.append(x)

In [29]:
accuracy_score(gt,preds)

0.2572347266881029

In [30]:
confusion_matrix(gt,preds)

array([[295, 322,  69,  56,  31],
       [370, 303,  52,  31,  58],
       [154, 355, 144,  90, 146],
       [ 93, 319, 187, 134, 130],
       [113, 228, 241, 189, 244]])

In [33]:
(295+322+370+303+52+355+144+90+187+134+130+189+244)/len(gt)

0.6465319246669728

In [ ]:
dfCEDEL[dfCEDEL.level!='B1'].drop('prediction',axis=1)

In [ ]:
dfCEDEL[dfCEDEL.level!='C2'].drop(['prediction'],axis=1)

In [ ]:
learnXLMRoberta.predict(' !Mi planes para el futero es estudiar mas espanol!')

In [ ]:
learnXLMRoberta.predict(' ¡Mi planes para el futero es estudiar mas espanol!')

In [ ]:
learnXLMRoberta.predict(' ¡Mis planes para el futero son estudiar mas espanol!')

In [ ]:
learnXLMRoberta.predict('¡Mi plan para el futero es estudiar mas espanol!')

In [ ]:
learnXLMRoberta.predict('¡Mi plan para el futuro es estudiar mas español!')

In [ ]:
learnXLMRoberta.predict('¡Mi plan para el futuro es estudiar más español!')

In [ ]:
learnXLMRoberta.predict('Ella es muy mal actriz')

In [ ]:
learnXLMRoberta.predict('Ella es muy mala actriz')

In [ ]:
learnXLMRoberta.predict('Ellos es amigos')

In [ ]:
learnXLMRoberta.predict('Ellos son amigos')

In [ ]:
learnXLMRoberta.predict('Boy haber una casa')

In [ ]:
learnXLMRoberta.predict('Voy haber una casa')

In [ ]:
learnXLMRoberta.predict('Voy a ber una casa')

In [ ]:
learnXLMRoberta.predict('Boy a ber una casa')

In [ ]:
learnXLMRoberta.predict('Voy a ber una casa')

In [ ]:
learnXLMRoberta.predict('Voy a ver una casa')

In [ ]:
learnXLMRoberta.predict('Me olvidé comprar pan')

In [ ]:
learnXLMRoberta.predict('Me olvidé de comprar pan')

In [ ]:
dfCEDEL[dfCEDEL.level!='C2']

In [ ]:
dfCEDEL[(dfCEDEL.level=='C2')&(dfCEDEL.predictionSencences=='A2')]

In [34]:
dfCEDEL

,L1,Proficiency,Text,level,prediction,predictionSencences,predictionIndSencences
0,English,Lower beginner,Me llamo es Helena. Ahora mismo soy estudiante en universidad Georgia State. Yo soy trabajadora en el restaurante. Yo soy mucho interesante en espanol. Yo quiero aprender hablar espanol con mi amigos. !Mi planes para el futero es estudiar mas espanol!,A1,A2,A1,"[B1, B2, A1, A1, A2, A1]"
1,English,Lower beginner,"La Region donde vive es EE.UU. Tengo vive en Georgia. Me mama y papa viven en Georgia. EE.UU es viver muy bien. Diferente regions mucho calor y otros mucho frio. En inverino, Georgia es frio. Inverino es diciembre y februro. Prefiere el inverino porque los holidays y me cumpleonous es en diciembre.",A1,A2,A1,"[A2, B2, A1, A1, B1, A1, A2, A2, A1, A1, A1]"
2,English,Lower beginner,Hola! Como Esta? <br/> !No hablo espanol! Muy dicifil! Jennifer Lopez es muy talentoso. Ella es vive en norteamericano con Senor Anthony.Senora Lopez tiene muy amigos y amigas famosa en Holllywood. Ella canta es muy mal. !Muy Mal! Jennifer Lopez es bonita y delgada y interesante. Ella es tanto. Es de donde Nueva York. Ella es muy mal actriz.,A1,A2,A2,"[A2, A1, A2, A1, A2, A2, A2, A1, A1]"
3,English,Lower beginner,Pienso que la nueva ley anti tabacco no es correcta en todo. Creo que desde que somos un contry libre que debemos ser capaces de hacer lo que queremos. Que medios que si personas quieren fumar y matar thereselves entonces usted los debe permitir. Por qué debe salimos de nuestra manera de cerciorarse ellos permanecen sano. Pienso que la ley es muy muda. Soy un creyente fuerte que usted debe ser capaz de hacer lo que usted quiere. Esta nueva ley le empuja un liitle a lejos. No a muchas personas ser án apenas capaz de dejar de fumar porque alguien los dice también. ¿Tomará tiempo para ellos d...,A1,B2,B2,"[B2, B2, B2, B2, A1, B2, B2, B2, B2, B2, B2, B2, B2, B2, A1]"
4,English,Lower beginner,"Hola, Me llamo Brittnay! yo tengo dos personas famosa! Mi favorita es Tyra Banks. Ella es la primera negrea modelo en todo el mundo.Ella es muy bonita y tienes un grande crozon. Ella ayuda mucho con el ninos con no padres . Mucho estrellas ayuda nada, pero no Tyra. Ella das su corzon todo! Tyra tambien ayuda mi! Tyra das mi familia 10,000 dolares para un nuevo casa. Mi familia nunca tiene dinero. Mi papa y mi mama tienes muchos problemas con uds.salud! Mi papa no tienes manos y mi mama no tienes tambien. Yo trababjo much para mi familia. Nosotros no tengo carro y yo nunca tiene un casa! Yo...",A1,A2,A2,"[A2, A2, A1, A2, A2, A2, B1, A1, A1, A1, A2, A1, A2, A2, A2, A2, A2, B1, A2, B1, B1, A2]"
...,...,...,...,...,...,...,...
295,German,Upper advanced,"La película sin sonido en negra y blanca se trata de Charles Chaplin. En ella encuentra a un bebé en la calle. Primero piensa que se trata del bebé de una mujer que ha entrado una casa cercana, pero al poner el bebé en el cochecito, la mujer sale del edificio y dice que no es su bebé. Él intenta poner el bebé en su lugar original pero un policía le ve y por eso levanta al bebé otra vez y se va. Ve a un hombre y decide darle el bebé sin decir algo y salir corriendo. El hombre está sorprendido y sigue a Charles. No puede encontrarle pero ve un cochecito enfrente de una tienda. Pone el bebé e...",C2,C1,C1,"[C1, C1, C1, C1, C1, C1, B1, B1, C1, B1, B1, C1, A1]"
296,English,Upper advanced,"Mi futuro. El qué hacer profesional. La verdad es que ni tengo idea. ¿Por qué usar fuentes adicionales o hacer investigaciones para una composición que no tiene una respuesta definida? Mis planes para el futuro me suena ya un poco a esas composiciones de primaria, cuando el mundo era todavía una ostra -si es que funciona esa frase en español- y tus esperanzas y sueños eran igualmente ilimitadas. Ahora, ya en mi tercera década de vida, me siento como si fuera uno de esos transeúntes ambulantes que vagabundea por las calles, ebrio, sin dirección fija, perdido. En realidad, mis planes para el...",C2,B2,B2,"[B2, B2, B1, B2, C1

In [35]:
learnXLMRoberta.predict('Chaplin estaba paseando por la calle')

[{'label': 'B',
  'score': 0.998668909072876,
  'class_index': 2,
  'class_labels': ['A1', 'A2', 'B1', 'B2', 'C1'],
  'probs': [9.217358638125006e-06,
   0.0005502514541149139,
   0.998668909072876,
   0.0003599120245780796,
   0.0004118206852581352]}]

In [36]:
learnXLMRoberta.predict('Chaplin estaba paseando para la calle')

[{'label': 'B',
  'score': 0.9942282438278198,
  'class_index': 2,
  'class_labels': ['A1', 'A2', 'B1', 'B2', 'C1'],
  'probs': [1.0992446732416283e-05,
   0.003311550710350275,
   0.9942282438278198,
   0.0015082433819770813,
   0.0009409052436240017]}]

In [42]:
learnXLMRoberta.predict('Hoy soy aburrida')

[{'label': 'A',
  'score': 0.9992579817771912,
  'class_index': 0,
  'class_labels': ['A1', 'A2', 'B1', 'B2', 'C1'],
  'probs': [0.9992579817771912,
   0.0006185397505760193,
   6.311406468739733e-05,
   4.677825199905783e-05,
   1.3514949387172237e-05]}]